# 동적 필드
- 컬렉션의 스키마에 정의된 모든 필드는 삽입할 엔티티에 포함되어야 함
- 일부 필드를 선택 사항으로 사용하려면 동적필드를 활성화하는 것이 좋음

참고 자료: https://milvus.io/docs/ko/enable-dynamic-field.md

## 개요
- $meta라는 이름의 예약된 필드이며 JSON 유형
- 스키마에 정의되지 않은 엔티티의 모든 동적 필드는 이 예약된 JSON 필드에 키-값 쌍으로 저장됨

## 동적 필드 활성화

In [1]:
from pymilvus import MilvusClient

client= MilvusClient(uri="http://localhost:19530")

if client.has_collection("my_collection"):
    client.drop_collection("my_collection")

client.create_collection(
    collection_name="my_collection",
    dimension=5,
    enable_dynamic_field=True
)

## 데이터 삽입

In [ ]:
data=[
    {"id": 0, "vector": [0.3580376395471989, -0.6023495712049978, 0.18414012509913835, -0.26286205330961354, 0.9029438446296592], "color": "pink_8682"},
    {"id": 1, "vector": [0.19886812562848388, 0.06023560599112088, 0.6976963061752597, 0.2614474506242501, 0.838729485096104], "color": "red_7025"},
    {"id": 2, "vector": [0.43742130801983836, -0.5597502546264526, 0.6457887650909682, 0.7894058910881185, 0.20785793220625592], "color": "orange_6781"},
    {"id": 3, "vector": [0.3172005263489739, 0.9719044792798428, -0.36981146090600725, -0.4860894583077995, 0.95791889146345], "color": "pink_9298"},
    {"id": 4, "vector": [0.4452349528804562, -0.8757026943054742, 0.8220779437047674, 0.46406290649483184, 0.30337481143159106], "color": "red_4794"},
    {"id": 5, "vector": [0.985825131989184, -0.8144651566660419, 0.6299267002202009, 0.1206906911183383, -0.1446277761879955], "color": "yellow_4222"},
    {"id": 6, "vector": [0.8371977790571115, -0.015764369584852833, -0.31062937026679327, -0.562666951622192, -0.8984947637863987], "color": "red_9392"},
    {"id": 7, "vector": [-0.33445148015177995, -0.2567135004164067, 0.8987539745369246, 0.9402995886420709, 0.5378064918413052], "color": "grey_8510"},
    {"id": 8, "vector": [0.39524717779832685, 0.4000257286739164, -0.5890507376891594, -0.8650502298996872, -0.6140360785406336], "color": "white_9381"},
    {"id": 9, "vector": [0.5718280481994695, 0.24070317428066512, -0.3737913482606834, -0.06726932177492717, -0.6980531615588608], "color": "purple_4976"}
]

res = client.insert(
    collection_name="my_collection",
    data=data
)

print(res)

{'insert_count': 10, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]}


## 동적 필드에서 스칼라 필드 인덱싱 하기
- 동적 필드를 활성화하면 정의되지 않은 모든 스칼라 필드가 JSON 형식의 키-값 쌍으로 저장
- Milvus는 이러한 정의되지 않은 스칼라 필드에 대한 인덱스 생성을 지원하며, JSON 경로 인덱스를 효과적으로 구축할 수 있음
1. 색인하려는 동적 필드 키 선택 예시로 "color"
2. 해당 키에서 찾은 값에 대한 형 변환 유형을 결정, Milvus는 동적 필드를 파싱하여 지정된 키 아래의 값을 추출한 다음 사용자가 구성한 유형으로 캐스팅
- 지원되는 json_cast_type 값은 bool(또는 BOOL), double(또는 DOUBLE), varchar(또는 VARCHAR)
- 구문 분석 또는 형 변환에 실패하면 해당 행은 인덱스에서 건너 뜀
3. 해당 키의 JSON 경로를 json_path로 지정, 동적 필드는 JSON으로 저장되므로 "color"와 같이 지정하거나 중첩된 구조가 있는 경우 더 깊은 경로(예: my_json["field"]["subfield"])를 지정
4. INVERTED 인덱스를 생성

In [3]:
index_parmas = client.prepare_index_params()

index_parmas.add_index(
    field_name="color",
    index_type="INVERTED",
    index_name="color_index",
    params={
        "json_path": "color",
        "json_cast_type": "varchar"
    }
)

client.create_index(
    collection_name="my_collection",
    index_params=index_parmas
)

# 동적 필드 쿼리 검색

In [5]:
query_vector = [0.3580376395471989, -0.6023495712049978, 0.18414012509913835, -0.26286205330961354, 0.9029438446296592]

res = client.search(
    collection_name="my_collection",
    data=[query_vector],
    limit=5,
    filter='color like "red%"',
    output_fields=["color"]
)

print(res)

data: [[{'id': 1, 'distance': 0.6290165185928345, 'entity': {'color': 'red_7025'}}, {'id': 4, 'distance': 0.5975796580314636, 'entity': {'color': 'red_4794'}}, {'id': 6, 'distance': -0.24996188282966614, 'entity': {'color': 'red_9392'}}]]
